<a href="https://colab.research.google.com/github/justinballas/Articles/blob/main/BICFR_Subaccount_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 17.5MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
%matplotlib inline

In [ ]:
carriers_list = ['LCN', 'NWD', 'PL', 'PRT', 'SYM']

carrier_dfs = {}
for carrier in carriers_list:
  df = pd.read_excel('Carrier Subaccounts Rankings Data.xlsx', sheet_name=carrier)
  carrier_dfs[carrier] = df
  

for carrier, df in carrier_dfs.items():
  df.rename(columns=lambda x: x.replace('\n',''), inplace=True)


columns = [0,1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]


mnstar_ratings = []
for carrier, df in carrier_dfs.items():
  temp_df = df.iloc[:, columns]
  temp_df.insert(loc=0, column='Carrier', value=carrier)
  mnstar_ratings.append(temp_df)


ratings_df = pd.concat(mnstar_ratings)


# replace_list = ["US Insurance ","US Fund ", "US VL Sub"]

# for term in replace_list:
#   ratings_df['Morningstar Category'] = ratings_df['Morningstar Category'].apply(lambda x: x.replace(term, ""))


agg_df_carrier = pd.pivot_table(ratings_df, index='Carrier', )

agg_df_class = pd.pivot_table(ratings_df, index='Morningstar Category')

fig = plt.figure(figsize=[15, 15])



#Creating Hsitogram
for carrier in enumerate(carrier_dfs.keys()):
  temp_df = ratings_df[ratings_df['Carrier']==carrier[1]]
  temp_df = temp_df[['Morningstar Rating Overall','Carrier']]
  ax = fig.add_subplot(2, 3, carrier[0]+1)
  title = temp_df.iloc[0,1]
  plt.ylim(0,120)
  plt.xlabel('Morningstar Overall',fontsize=15)
  plt.ylabel('Frequency',fontsize=15)
  plt.title(title)
  ax.hist(temp_df['Morningstar Rating Overall'])

plt.savefig('Carrier Histograms.png')


#Seperating into tables for different sheets
man_tenure = pd.merge(agg_df_carrier['Manager Tenure (Average)'], agg_df_carrier['Manager Tenure (Average)'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

sharpe = pd.merge(agg_df_carrier['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'], agg_df_carrier['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

exp_ratio = pd.merge(agg_df_carrier['Prospectus Net Expense Ratio'], agg_df_carrier['Prospectus Net Expense Ratio'].rank(), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))
ms_ratings_cols = ['Morningstar Rating 10 Yr', 'Morningstar Rating 5 Yr', 'Morningstar Rating 3 Yr', 'Morningstar Rating Overall']
ms_ratings =  agg_df_carrier.loc[:,ms_ratings_cols]



#--------------------------------------
for col in ms_ratings_cols:
  ms_ratings["{} Rank".format(col)] = ms_ratings[col].rank(ascending=False)

#--------------------------------------
rank_cat_cols = ['Total Ret % Rank Cat 1 Yr (Mo-End)', 'Total Ret % Rank Cat 3 Yr (Mo-End)', 'Total Ret % Rank Cat 5 Yr (Mo-End)', 'Total Ret % Rank Cat 10 Yr (Mo-End)']
rank_cat = agg_df_carrier.loc[:, rank_cat_cols]

for col in rank_cat_cols:
  rank_cat["{} Rank".format(col)] = rank_cat[col].rank()

#--------------------------------------
total_ret_cols = ['Total Ret 1 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 3 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 5 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 10 Yr (Mo-End) Base Currency']
total_ret = agg_df_carrier.loc[:, total_ret_cols]

for col in total_ret_cols:
  total_ret["{} Rank".format(col)] = total_ret[col].rank(ascending=False)


#Best in class fund selection
category_col = df['Morningstar Category']
asset_classes = category_col.unique()

ratings_df['sum_ratings'] = ratings_df['Morningstar Rating 10 Yr'] + ratings_df['Morningstar Rating 5 Yr'] + ratings_df['Morningstar Rating 3 Yr'] + ratings_df['Morningstar Rating Overall']

ratings_df['avg_rankings'] = (ratings_df['Total Ret % Rank Cat 1 Yr (Mo-End)'] + ratings_df['Total Ret % Rank Cat 3 Yr (Mo-End)'] + ratings_df['Total Ret % Rank Cat 5 Yr (Mo-End)'] + ratings_df['Total Ret % Rank Cat 10 Yr (Mo-End)'])/4


ratings_df['total_score'] = ratings_df['sum_ratings'] + ratings_df['avg_rankings'].apply(lambda x: 1-(x/100))

bicf_list = []
for carrier in carriers_list:
  temp_df = ratings_df[ratings_df['Carrier']==carrier]
  for asset_class in asset_classes:
    class_df = temp_df[temp_df['Morningstar Category']==asset_class]
    class_df['BICF'] = ['yes' if i == class_df['total_score'].max() else 'no' for i in class_df['total_score']]
    bicf_list.append(class_df)
    
ratings_df = pd.concat(bicf_list)

man_tenure = man_tenure.transpose()
ms_ratings = ms_ratings.transpose()
sharpe = sharpe.transpose()
exp_ratio = exp_ratio.transpose()
rank_cat = rank_cat.transpose()
total_ret = total_ret.transpose()


#writing to excel
xlwriter = pd.ExcelWriter('BICFR Subaccount Rankings.xlsx', engine='xlsxwriter')

man_tenure.to_excel(xlwriter, sheet_name='Manager Tenure')
ms_ratings.to_excel(xlwriter, sheet_name='Ratings')
sharpe.to_excel(xlwriter, sheet_name='Sharpe')
exp_ratio.to_excel(xlwriter, sheet_name='Expense Ratio')
rank_cat.to_excel(xlwriter, sheet_name='Return Rankings')
total_ret.to_excel(xlwriter, sheet_name='Total Return')

agg_df_class.to_excel(xlwriter, sheet_name='Asset Class Breakdown')

bicf_df = ratings_df[ratings_df['BICF']=='yes']
bicf_df.set_index(['Carrier','Morningstar Category'], inplace=True)
bicf_df.to_excel(xlwriter, sheet_name='BICFR')

ratings_df.set_index(['Carrier', 'Name'], inplace=True)
ratings_df.to_excel(xlwriter, sheet_name='All Data')

empty_sheet = pd.DataFrame()
empty_sheet.to_excel(xlwriter, sheet_name='Histogram')
worksheet=xlwriter.sheets['Histogram']
worksheet.insert_image('B2','Carrier Histograms.png')

list(carrier_dfs.values())[0].to_excel(xlwriter, sheet_name='LCN')
list(carrier_dfs.values())[1].to_excel(xlwriter, sheet_name='NWD')
list(carrier_dfs.values())[2].to_excel(xlwriter, sheet_name='PL')
list(carrier_dfs.values())[3].to_excel(xlwriter, sheet_name='PRT')
list(carrier_dfs.values())[4].to_excel(xlwriter, sheet_name='SYM')

xlwriter.save()

xlwriter.close()

In [ ]:
man_tenure.rename(columns={'Manager Tenure (Average) Rank':'Rank'})

In [ ]:
man_tenure.melt()

,Carrier,value
0,LCN,7.002277
1,LCN,5.000000
2,NWD,7.250821
3,NWD,4.000000
4,PL,7.563750
5,PL,2.000000
6,PRT,7.314184
7,PRT,3.000000
8,SYM,8.162356
9,SYM,1.000000


In [ ]:
ratings_df.describe()

,Total Ret 1 Yr (Mo-End) Base Currency,Total Ret Annlzd 3 Yr (Mo-End) Base Currency,Total Ret Annlzd 5 Yr (Mo-End) Base Currency,Total Ret Annlzd 10 Yr (Mo-End) Base Currency,Morningstar Rating Overall,Morningstar Rating 3 Yr,Morningstar Rating 5 Yr,Morningstar Rating 10 Yr,Total Ret % Rank Cat 1 Yr (Mo-End),Total Ret % Rank Cat 3 Yr (Mo-End),Total Ret % Rank Cat 5 Yr (Mo-End),Total Ret % Rank Cat 10 Yr (Mo-End),Manager Tenure (Average),Prospectus Net Expense Ratio,Sharpe Ratio 3 Yr (Mo-End) Risk Currency,Std Dev 3 Yr (Mo-End) Risk Currency,Yield to Maturity,Modified Duration,Total Ret YTD (Daily) Base Currency,Total Ret YTD (Mo-End) Base Currency,sum_ratings,avg_rankings,total_score
count,1065.000000,1047.000000,1029.000000,981.000000,1036.000000,1036.000000,1017.000000,969.000000,1058.000000,1040.000000,1021.000000,973.000000,1061.000000,1065.000000,1047.000000,1047.000000,116.000000,166.000000,1065.000000,1065.000000,969.000000,973.000000,969.000000
mean,47.035643,10.659054,11.088931,8.715708,3.396718,3.289575,3.323500,3.285862,48.310964,43.954808,42.512243,42.353546,7.486305,0.751484,0.625998,15.889026,2.179224,4.766627,6.730103,4.432742,13.369453,43.855087,13.931068
std,28.840705,6.494634,6.099669,4.576108,1.016743,1.053693,1.078863,1.008136,26.191975,27.487802,27.376301,26.332208,4.188393,0.306641,0.449709,7.256938,1.558099,3.170567,6.295908,6.121822,3.821391,22.499207,4.019073
min,-15.400000,-4.840000,-0.400000,-6.510000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.330000,0.030000,-4.720000,0.230000,-0.980000,0.050000,-13.960000,-14.730000,4.000000,1.250000,4.042500
25%,24.640000,5.590000,6.350000,5.300000,3.000000,3.000000,3.000000,3.000000,25.000000,21.000000,20.000000,20.000000,4.310000,0.580000,0.490000,10.660000,1.110000,2.940000,1.730000,0.260000,11.000000,26.500000,11.497500
50%,50.430000,9.990000,11.000000,8.880000,3.000000,3.000000,3.000000,3.000000,47.500000,42.000000,39.000000,39.000000,6.790000,0.770000,0.650000,17.920000,1.960000,3.640000,6.350000,3.200000,13.000000,42.750000,13.712500
75%,64.770000,14.445000,14.870000,12.060000,4.000000,4.000000,4.000000,4.000000,71.000000,67.000000,63.000000,64.000000,9.750000,0.930000,0.855000,20.700000,2.800000,5.637500,11.240000,8.020000,16.000000,59.750000,16.707500
max,126.020000,40.330000,35.160000,22.860000,5.000000,5.000000,5.000000,5.000000,100.000000,100.000000,100.000000,100.000000,23.750000,2.160000,1.470000,31.970000,6.300000,19.090000,25.660000,26.500000,20.000000,95.750000,20.987500
